# Metadata和权限体系

## 一、理论介绍

在RAG（Retrieval-Augmented Generation）中，元数据（metadata）和权限体系是确保信息安全和有效管理的重要组成部分。元数据提供了关于数据的上下文信息，例如数据的来源、创建时间、修改历史等，这些信息有助于用户理解和使用数据。

权限体系则是对数据访问和操作的控制机制，确保只有授权用户才能访问特定的数据。通过定义不同用户角色和权限，系统可以有效地管理数据的安全性，防止未授权访问和数据泄露。

在RAG的应用中，合理的元数据管理和权限控制能提高数据的可用性，还能增强系统的安全性，确保用户在获取信息时能够遵循相关的法律法规和公司政策。



## 二、代码实现

### 2.1 数据准备

In [1]:
import re
import json

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

from langchain_openai import ChatOpenAI
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate

import warnings
warnings.filterwarnings("ignore")


def clean_text(text: str):
    """
    实现文本清理函数

    参数:
        text: 需要清理的字段

    返回:
        清理完成后返回的字段
    
    """
    # 删除每页开头与结尾标语及链接
    text = re.sub(r'→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←', '', text)
    text = re.sub(r'→_→\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n←_←', '', text)
    # 删除字符串开头的空格
    text = re.sub(r'\s+', '', text)
    # 删除回车
    text = re.sub(r'\n+', '', text)

    return text

In [2]:
pdf_path = "data/pumpkin_book.pdf"
embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')

chunk_size=5000
chunk_overlap=1000

In [3]:
# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径，加载PDF
loader = PyMuPDFLoader(pdf_path)

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

# 对内容进行一下清洗
data_pages = pdf_pages[13:-13]
for page in data_pages:
    page.page_content = clean_text(page.page_content)

# 文档分块
text_splitter = CharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator='')

split_docs = text_splitter.split_documents(data_pages)

### 2.2 查看/修改metadata

In [4]:
# 选择其中一个分片查看metadata，现在的metadata为分片过程中默认内容
split_docs[4].metadata

{'source': 'data/pumpkin_book.pdf',
 'file_path': 'data/pumpkin_book.pdf',
 'page': 17,
 'total_pages': 196,
 'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'xdvipdfmx (20200315)',
 'creationDate': "D:20230303170709-00'00'",
 'modDate': '',
 'trapped': ''}

In [5]:
# 修改分块文档的metadata，加入chunk_size 和 chunk_overlap
for doc in split_docs:
    doc.metadata['chunk_size'] = chunk_size
    doc.metadata['chunk_overlap'] = chunk_overlap

### 2.3 权限体系

实际使用中，我们经常使用metadata来设置权限体系，从而实现不同权限用户的数据隔离。下面我们将用上述数据集来模拟权限体系的操作。

假定：本书的50-60页为敏感页面，需要访问权限为2才能进行访问。

In [6]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    使用 OpenAI 嵌入将 PDF 书籍编码为向量存储。

    参数:
        path: PDF 文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    返回:
        包含内容的向量存储。
    """

    # 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径，加载PDF
    loader = PyMuPDFLoader(path)
    
    # 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
    pdf_pages = loader.load()
    
    # 第13页为南瓜书第一页正文，因此从13页开始,从倒数13页涉及敏感用语，因此从-13页结束
    data_pages = pdf_pages[13:-13]

    for page in data_pages:
        page.page_content = clean_text(page.page_content)

    # 文档分块
    text_splitter = CharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator='')

    split_docs = text_splitter.split_documents(data_pages)

    # 修改metadat
    for doc in split_docs:
        # 加入chunk_size和chunk_overlap
        doc.metadata['chunk_size'] = chunk_size
        doc.metadata['chunk_overlap'] = chunk_overlap
        # 加入数据访问权限
        doc.metadata['data_level'] = 2 if doc.metadata['page'] >= 50 and doc.metadata['page'] <= 60 else 1

    # 构建向量库
    vectordb = Chroma.from_documents(documents=split_docs, embedding=embedding)

    return vectordb

In [7]:
# 将pdf放入到向量数据库中
vectordb = encode_pdf(pdf_path, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [8]:
# 第一个人是权限较高，拥有数据权限为2，可以查看全部数据库
hypothetical_doc = "神经元模型"
similar_docs = vectordb.similarity_search(hypothetical_doc, k=3)
similar_docs

# 召回top3分别是 53、57、30页

[Document(metadata={'author': '', 'chunk_overlap': 1000, 'chunk_size': 5000, 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'data_level': 2, 'file_path': 'data/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 53, 'producer': 'xdvipdfmx (20200315)', 'source': 'data/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='第5章神经网络神经网络类算法可以堪称当今最主流的一类机器学习算法，其本质上和前几章讲到的线性回归、对数几率回归、决策树等算法一样均属于机器学习算法，也是被发明用来完成分类和回归等任务。不过由于神经网络类算法在如今超强算力的加持下效果表现极其出色，且从理论角度来说神经网络层堆叠得越深其效果越好，因此也单独称用深层神经网络类算法所做的机器学习为深度学习，属于机器学习的子集。5.1神经元模型本节对神经元模型的介绍通俗易懂，在此不再赘述。本节第2段提到“阈值”(threshold)的概念时，“西瓜书”左侧边注特意强调是“阈(yù)”而不是“阀(fá)”，这是因为该字确实很容易认错，读者注意一下即可。图5.1所示的M-P神经元模型，其中的“M-P”便是两位作者McCulloch和Pitts的首字母简写。5.2感知机与多层网络5.2.1式(5.1)和式(5.2)的推导此式是感知机学习算法中的参数更新公式，下面依次给出感知机模型、学习策略和学习算法的具体介绍[1]：感知机模型：已知感知机由两层神经元组成，故感知机模型的公式可表示为y=fnXi=1wixi−θ!=f(wTx−θ)其中，x∈Rn，为样本的特征向量，是感知机模型的输入；w,θ是感知机模型的参数，w∈Rn，为权重，θ为阈值。假定f为阶跃函数，那么感知机模型的公式可进一步表示为（用ε(·)代表阶

In [9]:
# 第一个人是权限较低，拥有数据权限为1，只能查看对应部分数据
hypothetical_doc = "神经元模型"
similar_docs = vectordb.similarity_search(hypothetical_doc, k=3, filter={"data_level": 1})
similar_docs

# 召回top3分别是 30、177、74页

[Document(metadata={'author': '', 'chunk_overlap': 1000, 'chunk_size': 5000, 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'data_level': 1, 'file_path': 'data/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 30, 'producer': 'xdvipdfmx (20200315)', 'source': 'data/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='第3章线性模型作为“西瓜书”介绍机器学习模型的开篇，线性模型也是机器学习中最为基础的模型，很多复杂模型均可认为由线性模型衍生而得，无论是曾经红极一时的支持向量机还是如今万众瞩目的神经网络，其中都有线性模型的影子。本章的线性回归和对数几率回归分别是回归和分类任务上常用的算法，因此属于重点内容，线性判别分析不常用，但是其核心思路和后续第10章将会讲到的经典降维算法主成分分析相同，因此也属于重点内容，且两者结合在一起看理解会更深刻。3.1基本形式第1章的1.2基本术语中讲述样本的定义时，我们说明了“西瓜书”和本书中向量的写法，当向量中的元素用分号“;”分隔时表示此向量为列向量，用逗号“,”分隔时表示为行向量。因此，式(3.2)中w=(w1;w2;...;wd)和x=(x1;x2;...;xd)均为d行1列的列向量。3.2线性回归3.2.1属性数值化为了能进行数学运算，样本中的非数值类属性都需要进行数值化。对于存在“序”关系的属性，可通过连续化将其转化为带有相对大小关系的连续值；对于不存在“序”关系的属性，可根据属性取值将其拆解为多个属性，例如“西瓜书”中所说的“瓜类”属性，可将其拆解为“是否是西瓜”、“是否是南瓜”、“是否是黄瓜”3个属性，其中每个属性的取值为1或0，1表示“是”，0表示“否”。具体地，假如现有3个瓜类样本：x1=(甜度=高;瓜